In [1]:
import sympy as sp

In [2]:
# Variable dummy Nc (numero de componentes)
nc = sp.symbols("Nc", integer=True)

# Indices
# Para derivadas de n
i = sp.symbols("i", cls=sp.Idx)
j = sp.symbols("j", cls=sp.Idx)

# Indices para variables
k = sp.symbols("k", cls=sp.Idx)
l = sp.symbols("l", cls=sp.Idx)

# Variables
n = sp.IndexedBase("n", shape=(nc,))
t = sp.symbols("T")
v = sp.symbols("V")


r = sp.IndexedBase('r')
q = sp.IndexedBase('q')

a = sp.IndexedBase('a')
b = sp.IndexedBase('b')


In [3]:
phi = n[k] * r[k] / sp.summation(n[l] * r[l], (l, 1, nc))

phi

n[k]*r[k]/Sum(n[l]*r[l], (l, 1, Nc))

In [4]:
exp = sp.diff(phi, n[i])

exp

KroneckerDelta(i, k)*r[k]/Sum(n[l]*r[l], (l, 1, Nc)) - n[k]*r[k]*Sum(KroneckerDelta(i, l)*r[l], (l, 1, Nc))/Sum(n[l]*r[l], (l, 1, Nc))**2

In [6]:
def simplify_kronecker(expr):
    # Primero evaluamos sumatorias con deltas adentro
    expr = expr.replace(
        lambda x: isinstance(x, sp.Sum) and any(isinstance(arg, sp.KroneckerDelta) for arg in sp.preorder_traversal(x.function)),
        lambda x: simplify_sum_with_kronecker(x)
    )
    # Luego transformamos deltas sueltos en Piecewise
    # expr = expr.replace(
    #     sp.KroneckerDelta,
    #     lambda i, j: sp.Piecewise((1, sp.Eq(i, j)), (0, True))
    # )

    return expr

def simplify_sum_with_kronecker(sum_expr):
    """Transforma sumatorias del estilo Sum(δ_il * f(l), (l, 1, Nc)) en f(i)"""
    idx = sum_expr.variables[0]
    f = sum_expr.function

    if not isinstance(f, sp.Mul):
        return sum_expr

    # Buscar sp.KroneckerDelta en el producto
    for arg in f.args:
        if isinstance(arg, sp.KroneckerDelta):
            delta = arg
            other_terms = sp.Mul(*[a for a in f.args if a != delta])
            i, j = delta.args
            # Determinar cuál es la variable de suma
            if i == idx:
                return other_terms.subs(idx, j)
            elif j == idx:
                return other_terms.subs(idx, i)

    return sum_expr


In [8]:
dedn = simplify_kronecker(exp)

In [9]:
dedn2 = sp.diff(dedn, n[j])

In [12]:
dedn2 = simplify_kronecker(dedn2)

In [15]:
dedn2

-KroneckerDelta(i, k)*r[j]*r[k]/Sum(n[l]*r[l], (l, 1, Nc))**2 - KroneckerDelta(j, k)*r[i]*r[k]/Sum(n[l]*r[l], (l, 1, Nc))**2 + 2*n[k]*r[i]*r[j]*r[k]/Sum(n[l]*r[l], (l, 1, Nc))**3

In [16]:
dedn2.subs(k, i).subs(sp.KroneckerDelta(i, j), 0)

2*n[i]*r[i]**2*r[j]/Sum(n[l]*r[l], (l, 1, Nc))**3 - r[i]*r[j]/Sum(n[l]*r[l], (l, 1, Nc))**2